In [1]:
import re
from tqdm import tqdm_notebook
import json

## Перевод из кирилицы в нашу транскрипцию

In [2]:
vowels_semi_pltl = {"ӓ":"a","ӭ":"e","ҍ":""}
vowels_long_pltl = {"я̄":"aa","е̄":"ee","ю̄":"uu","ӣ":"ii"}
vowels_pltl = {"я":"a","е":"e","ю":"u","ё":"o","ь":"","и":"i"}
vowels_semi_pltl = {"ӓ":"a","ӭ":"e","ҍ":""}
vowels_long_pltl = {"я̄":"aa","е̄":"ee","ю̄":"uu","ӣ":"ii"}
vowels_pltl = {"я":"a","е":"e","ю":"u","ё":"o","ь":"","и":"i"}
vowels_long = {"я̄":"jaa","е̄":"jee","ю̄":"juu","э̄":"ee","ӯ":"uu","а̄":"aa","ӣ":"ii","о̄":"oo"}
vowels = {"ё":"jo","а":"a","ы":"ɨ","у":"u","э":"e","я":"ja","е":"je","ю":"ju","и":"i","о":"o","ӭ":"e","ӓ":"a"}
cnsnt_pltl = {"дз":"ʒ","н":"ɲ","ћ":"h","б":"b","в":"v","г":"g","д":"d","ж":"ž","з":"z","к":"k","л":"l","ӆ":"l̥","м":"m","ӎ":"m̥","ӊ":"n̥","ӈ":"ŋ","п":"p","р":"r","ҏ":"r̥","с":"s","т":"t","ф":"f","х":"x","һ":"h","ш":"š","ц":"c"}
cnsnt_no_pltl = {"дж":"ǯ", "й":"j", "ҋ":"j̥", "ч":"č"}
cnsnts = {"дж":"ǯ","дз":"ʒ","ћ":"h","ҋ":"j̥","й":"j","б":"b","в":"v","г":"g","д":"d","ж":"ž","з":"z","к":"k","л":"l","ӆ":"l̥","м":"m","ӎ":"m̥","н":"n","ӊ":"n̥","ӈ":"ŋ","п":"p","р":"r","ҏ":"r̥","с":"s","т":"t","ф":"f","х":"x","һ":"h","ш":"š","ц":"c","ч":"č","ҍ":"","ь":"","ъ":""}

def get_transcript(word):
    if word == '':
        return ''
    else:
        # заменяем J:
        word = re.sub(r"J", r"j̥", word)
        # делаем все лоуверкейс
        word = word.lower()
        # заменяем полумягкие н:
        for vowel, new_vowel in vowels_semi_pltl.items():
            # сначала долгие, потому что это два символа:
            word = re.sub("нн" + vowel, r"n'n'" + new_vowel, word)
            word = re.sub("н" + vowel, r"n'" + new_vowel, word)
        # заменяем палатальный л:
        word = re.sub(r"льй(\s|$)", r"lij\1", word)
        word = re.sub(r"льй", r"l'j", word)
        # гласные, перед которыми мягкость:
        #     сначала долгие, потому что это два символа на самом деле:
        for vowel, new_vowel in vowels_long_pltl.items():
            for cnsnt, new_cnsnt in cnsnt_pltl.items():
                word = re.sub(cnsnt + cnsnt + vowel, new_cnsnt + "'" + new_cnsnt + "'" + new_vowel, word)
                word = re.sub(cnsnt + vowel, new_cnsnt + "'" + new_vowel, word)
            for cnsnt, new_cnsnt in cnsnt_no_pltl.items():
                word = re.sub(cnsnt + cnsnt + vowel, new_cnsnt + new_cnsnt + new_vowel, word)
                word = re.sub(cnsnt + vowel, new_cnsnt + new_vowel, word)
        #     обычные гласные, перед которыми мягкость:
        for vowel, new_vowel in vowels_pltl.items():
            for cnsnt, new_cnsnt in cnsnt_pltl.items():
                word = re.sub(cnsnt + cnsnt + vowel, new_cnsnt + "'" + new_cnsnt + "'" + new_vowel, word)
                word = re.sub(cnsnt + vowel, new_cnsnt + "'" + new_vowel, word)
            for cnsnt, new_cnsnt in cnsnt_no_pltl.items():
                word = re.sub(cnsnt + cnsnt + vowel, new_cnsnt + new_cnsnt + new_vowel, word)
                word = re.sub(cnsnt + vowel, new_cnsnt + new_vowel, word)
        # заменяем оставшиеся долгие гласные:
        for vowel, new_vowel in vowels_long.items():
            word = re.sub(vowel, new_vowel, word)
        # оставшиеся гласные:
        for vowel, new_vowel in vowels.items():
            word = re.sub(vowel, new_vowel, word)
        # оставшиеся согласные:
        for cnsnt, new_cnsnt in cnsnts.items():
                word = re.sub(cnsnt, new_cnsnt, word)
        return word

## Переводим словарь в json и csv

In [3]:
with open("../sa-ru-kuruch.dsl", encoding="utf-8") as f:
    kuruch = f.read()
kuruch = kuruch.splitlines()[3:] # [3:] - обрезаем сверху метаданные

In [4]:
kuruch_list = []
catch_lower = False
# strangers = []
n = -1
for line in tqdm_notebook(kuruch):
    if not re.match(r'\t', line):
        # в начале статьи нет табулиции
        if line.islower():
            # дериваты, с ними работаем отдельно в каждой статье, нет смысла отдельно разбирать
            catch_lower = True
            continue
        main_word = re.sub(r'\d',r'', line.lower()).strip()
        transcribed_main_word = get_transcript(main_word)
    elif line == '\t[m1][/m]':
        continue
    elif catch_lower:
        catch_lower = False
    elif re.match(r'\t\[m0\]', line):
        continue
    elif re.match(r'\t\[m1\][а-яА-я ]+\[\/m\]', line):
#         strangers.append({'line': line, 'number': n})
        continue
    elif re.match(r'\t\[m1\]([^[ ].+?)(\[b\].*?\[\/b\].*?)?\[/m\]', line) and not re.match('\t\[m1\]([^[ ].+?)\[/b]', line):
        match = re.search(r'\t\[m1\](.*?)(\[b\].*?\[\/b\].*?)?;\[/m\]', line)
        try:
            lost_piece, more_examples = match.groups('')
        except AttributeError:
#             strangers.append({'line': line, 'number': n})
            continue
            
        kuruch_list[-1]['definitions'].append(lost_piece)
        
        if more_examples != '':
            example, definition = re.search(r'\[b\](.*?)\[\/b\](.*?)$', more_examples).groups('')
            example = example.strip()
            item_dict = {'example_cyrillic': example, 
                         'example_transcription': get_transcript(example), 
                         'example_definition': definition.strip()}
            kuruch_list[-1]['examples'].append(item_dict)
    else:
        regex = r'\[m1\] ?(?:\[b\])?(.*?)\[\/b\] ?(?:\(\[b\](.*?)\[\/b\]\))? ?\d? ?(о\. ч\. )?([IV]{1,3})?(?:, (\d.))? ?(.*?)(?:\[/m\])?$'
        match = re.search(regex, line)
        try:
            word, form, common_number, word_type, word_alt, expanded_definition = match.groups('')
        except AttributeError:
#             strangers.append({'line': line, 'number': n})
            continue
        
        word = re.sub(r'\[.*?\]', r'', word).lower().strip()
        if ' ' in word:
            regex = r'(.*?)(?:\d\.(.*?))?$'
            example_definition, word_definition = re.search(regex, expanded_definition).groups('')
            item_dict = {'example_cyrillic': word, 
                             'example_transcription': get_transcript(word), 
                             'example_definition': example_definition.strip()}
            kuruch_list[-1]['examples'].append(item_dict)
            if word_definition.strip() != '':
                kuruch_list[-1]['definitions'].append(word_definition.strip())
            continue
            
        transcribed_word = get_transcript(word)
        
        if common_number != '':
            common_number = 'общее число'
        
        definitions = []
        examples = []
        for item in re.findall(r'(?:\d\. )?(.*?)(?:;\s?|\d\. |$|\\#\s?)', expanded_definition):
            if '[/b]' in item:
                example, definition = re.search(r'(?:\[b\])?(.*?)\[\/b\](.*?)$', item).groups('')
                example = example.strip()
                item_dict = {'example_cyrillic': example, 
                             'example_transcription': get_transcript(example), 
                             'example_definition': definition.strip()}
                examples.append(item_dict)
            else:
                if item.strip() != '':
                    definitions.append(item.strip())
                    
        if word_type != '':
            if transcribed_word.endswith('e'):
                pos = 'Verb'
            else:
                pos = 'Noun'
        else:
            pos = ''
        
        instance = {'cyrillic_backwards': word[::-1],
                    'transcription_backwards': transcribed_word[::-1],
                    'cyrillic': word,
                    'transcription': transcribed_word,
                    'derived_cyrillic': main_word,
                    'derived_transcription': transcribed_main_word,
                    'form_cyrillic': form,
                    'form_transcription': get_transcript(form),
                    'pos': pos,
                    'common_number': common_number,
                    'type': word_type,
                    'type_alternation': word_alt,
                    'definitions': definitions,
                    'examples' : examples
                    }
        
        kuruch_list.append(instance)
        n += 1

In [5]:
# первый раз

# for stranger in strangers:
#     print(stranger, end='\n\n')
#     print(kuruch_list[stranger['number']]['definitions'])
#     print(kuruch_list[stranger['number']]['examples'])
#     decision = str(input())
#     stranger['decision'] = decision
# with open('../intermediate files/strangers.json', 'w', encoding='utf-8') as json_file:
#     json.dump(strangers, json_file, ensure_ascii=False)

# не первый раз
with open('../intremediate files/strangers.json', encoding='utf-8') as f:
    strangers = json.load(f)

In [ ]:
for stranger in strangers:
    n = stranger['number']
    if 'decision' not in stranger:
        print(stranger['line'])
        print(kuruch_list[n])
        stranger['decision'] = str(input())
    if stranger['decision'] == 'part of last definition':
        part = re.search(r'\t\[m1\](.*?)\[\/m\]', stranger['line']).group()
        kuruch_list[n]['definitions'][-1] += ' ' + part
    elif stranger['decision'] == 'part of last definition and new definition':
        lost_part, new_def = re.search(r'\t\[m1\](.*?);? ?\d\. (.*?)\[\/m\]', stranger['line']).groups('')
        kuruch_list[n]['definitions'][-1] += ' ' + lost_part
        kuruch_list[n]['definitions'].append(new_def.strip())
    elif stranger['decision'] == 'part of last example definition':
        part = re.search(r'\t\[m1\](.*?)\[\/m\]', stranger['line']).group()
        kuruch_list[n]['examples'][-1]['example_definition'] += ' ' + part
    elif stranger['decision'] == 'new definition for last item':
        part = re.search(r'\t\[m1\] ?\d\. (.*?)\[\/m\]', stranger['line']).group()
        kuruch_list[n]['definitions'].append(part)
    elif stranger['decision'] == 'new definitions for last item':
        parts = re.findall(r'\d\. (.*?)(?:\[\/m\]|\d)', stranger['line'])
        for part in parts:
            kuruch_list[n]['definitions'].append(part.strip())
    elif stranger['decision'] == 'part of last definition and new definitions':
        lost_part, new = re.search(r'\t\[m1\](.*?)(\d\..*?)\[\/m\]', stranger['line']).groups('')
        kuruch_list[n]['definitions'][-1] += ' ' + lost_part.strip()
        parts = re.findall(r'\d\. (.*?)(?:$|\d)', new)
        for part in parts:
            kuruch_list[n]['definitions'].append(part.strip())
    elif stranger['decision'] == 'part of last definition and new definitions and new example':
        print(stranger['line'])
        kuruch_list[n]['definitions'][-1] += ' ' + str(input('lost part: '))
        example = str(input('example: '))
        kuruch_list[n]['examples'].append({'example_cyrillic': example, 
                                             'example_transcription': get_transcript(example), 
                                             'example_definition': str(input('definition: '))})
    elif stranger['decision'] == 'part of last example and new definition':
        print(stranger['line'])
        kuruch_list[n]['examples'][-1]['example_definition'] += ' ' + str(input('lost part: '))
        new = str(input('new definition: '))
        kuruch_list[n]['definitions'].append(new)

	[m1]иногда, бывало); пудзэ чёлнэв чурьвэть чёххча олени обдирают рога осенью[/m]
lost part: иногда, бывало)
example: пудзэ чёлнэв чурьвэть чёххча
definition: олени обдирают рога осенью
	[m1]затишье 2. перен. успокаиваться / успокоиться (после шума, скандала)[/m]


In [9]:
kuruch_list[8]

{'cyrillic_backwards': 'эрбба',
 'transcription_backwards': 'erbba',
 'cyrillic': 'аббрэ',
 'transcription': 'abbre',
 'derived_cyrillic': 'аббьр',
 'derived_transcription': "ab'b'r",
 'form_cyrillic': '',
 'form_transcription': '',
 'pos': 'Verb',
 'common_number': '',
 'type': 'II',
 'type_alternation': '4 ',
 'definitions': ['идти - о дожде, дождить'],
 'examples': [{'example_cyrillic': 'ке̄сся тоаййв эбьрэ',
   'example_transcription': "k'ees's'a toajjv eb're",
   'example_definition': 'лётом часто шёл дождь'}]}

In [10]:
with open('../done/kuruch.csv','w',encoding='utf-8') as f:
    f.write('cyrillic_backwards\ttranscription_backwards\tcyrillic\ttranscription\tderived_cyrillic\tderived_transcription\tform_cyrillic\tform_transcription\tpos\tcommon_number\ttype\ttype_alternation\tdefinitions\texample_cyrillic\texample_transcription\texample_definition\n')
    for word in kuruch_list:
        examples = ''
        for example in word['examples']:
            examples += f"{example['example_cyrillic']}\t{example['example_transcription']}\t{example['example_definition']}\t"
        line = f"{word['cyrillic_backwards']}\t{word['transcription_backwards']}\t{word['cyrillic']}\t{word['transcription']}\t{word['derived_cyrillic']}\t{word['derived_transcription']}\t{word['form_cyrillic']}\t{word['form_transcription']}\t{word['pos']}\t{word['common_number']}\t{word['type']}\t{word['type_alternation']}\t{';'.join(word['definitions'])}\t{examples}\n"
        f.write(line)

In [11]:
with open('../done/kuruch.json', 'w', encoding='utf-8') as json_file:
    json.dump(kuruch_list, json_file, ensure_ascii=False)